In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Artist DB


In [4]:
indie_df = pd.read_csv('data/aritst_new_ver.csv')

In [5]:
indie_df

,code,artist,year
0,80023059,굴소년단,2008
1,7536,넬,2008
2,4419,노브레인,2008
3,80027619,마리서사,2008
4,71959,못,2008
...,...,...,...
188,20016929,최엘비,2022
189,20036652,THAMA,2022
190,80095880,이상의날개,2022
191,20119889,유다빈밴드,2022


In [11]:
indie_df.rename(columns = {'code': 'id'}, inplace=True)

In [12]:
indie_df

,id,artist,year
0,80023059,굴소년단,2008
1,7536,넬,2008
2,4419,노브레인,2008
3,80027619,마리서사,2008
4,71959,못,2008
...,...,...,...
188,20016929,최엘비,2022
189,20036652,THAMA,2022
190,80095880,이상의날개,2022
191,20119889,유다빈밴드,2022


In [13]:
indie_id_list = indie_df['id'].to_list()
indie_id_list

[80023059,
 7536,
 4419,
 80027619,
 71959,
 80001659,
 80009990,
 2472,
 10519,
 328,
 80003819,
 80022511,
 65291,
 65269,
 80023263,
 2149,
 80023888,
 80020976,
 21983,
 80021537,
 80026987,
 24347,
 3352,
 80057634,
 80042861,
 80054221,
 1529,
 80018503,
 22751,
 80055461,
 31236,
 80052013,
 80050837,
 80026982,
 80028770,
 6567,
 80060701,
 2499,
 80021125,
 2459,
 1915,
 8010698,
 80054221,
 80050909,
 80348286,
 80067149,
 80062575,
 80073013,
 6653,
 80023888,
 80080564,
 1660,
 80026982,
 14830,
 80063054,
 80027768,
 80021125,
 80063748,
 80070806,
 40150,
 80020042,
 80067149,
 80089306,
 80074007,
 80023263,
 80050546,
 80028770,
 53445,
 80091248,
 7573,
 80063054,
 80096263,
 80057634,
 10519,
 80054221,
 80051717,
 8008582,
 80070806,
 80055461,
 22688,
 80093531,
 80068124,
 80058413,
 80057634,
 80010025,
 80101412,
 80051717,
 8008582,
 80093531,
 80154056,
 80012738,
 80041839,
 80025855,
 80110244,
 10519,
 328,
 1443,
 2538,
 80153142,
 80110608,
 80074007,
 801

## Lyrics

In [14]:
%%time

def lyrics_scraping(artist_id):
    # 1. url 받아오기
    # def scrape_bugs(artist_id, page_numbers):
    # for page_number in range(1,page_numbers+1):

    url = f'https://music.bugs.co.kr/artist/{artist_id}/tracks?type=RELEASE'
    request = requests.get(url) # url 불러오기 요청
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')

    song_list = soup.find("table", attrs={'class':'list trackList'}) # 해당 연도의 곡 리스트
    # 해당 url에서 class가 list trackList인 'table' 태그의 데이터를 song_list에 담음

    titles = soup.select('p.title') # p 태그의 클래스가 title인 데이터
    artists = soup.select('p.artist') # p 태그의 클래스가 artist인 데이터
    albums = soup.select('td.left > a.album') # td 태그의 클래스가 left이고, 하위 데이터 중 a 태그의 클래스가 album인 데이터

    song_title = [] # 노래 제목
    song_artist = [] # 가수명
    album_list = [] # 앨범명
    like_counts = [] # 좋아요 수
    song_lyrics = [] # 가사
    id_url_list = [] # 노래 고유 번호가 있는 href 데이터를 담는 리스트
    id_list = [] # 노래 고유 번호를 담는 리스트


    for elem in soup.find_all('a', href=re.compile('https://music.bugs.co.kr/track')): # 해당 url에서 a 태그를 전부 찾기
        id_url_list.append(elem['href']) # a 태그 중에서 href 데이터만 따로 빼내서 id_url_list에 담기

    for j in range(len(id_url_list)):
        a = id_url_list[j].lstrip('https://music.bugs.co.kr/track/')  # 링크1 제거
        # lstrip() : 데이터의 왼쪽에서 괄호 안 데이터의 패턴이 발견되면 제거

        a1 = a.rstrip('wl_ref=list_tr_08_tr')  # 링크2 제거
        # rstrip() : 데이터의 오른쪽에서 괄호 안 데이터의 패턴이 발견되면 제거
        b = a1.rstrip('?')  # 물음표 제거

        id_list.append(b) 

    # 타이틀, 가수, 앨범명
    for k in range(len(id_list)):
        rank = k + 1 # k=0부터 시작

        title = titles[k].text.strip().split('\n')[0] # titles 데이터에서 양쪽 공백을 제거하고, \n을 기준으로 분리
        song_title.append(title) #song_title에 추가

        artist = artists[k].text.strip().split('\n')[0] # artists 데이터에서 양쪽 공백을 제거하고, \n을 기준으로 분리
        song_artist.append(artist) #song_artist에 추가

        album = albums[k].text.strip().split('\n')[0] # albums 데이터에서 양쪽 공백을 제거하고, \n을 기준으로 분리
        album_list.append(album) #album_list에 추가



    # 좋아요 & 가사 & 앨범 이미지(url)
    for m in range(len(id_list)):
        l_url = 'https://music.bugs.co.kr/track/' + str(id_list[m]) + '?wl_ref=list_tr_08_' # 전체 url 생성 : 곡 상세 페이지
        l_request = requests.get(l_url)
        l_html = l_request.text
        l_soup = BeautifulSoup(l_html, "html.parser")

        # 좋아요
        like_count = str(l_soup.select(
            '#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.etcInfo > span > a > span > em')) # 좋아요 데이터가 있는 html 구조
        like_count = re.sub('<.+?>', '', like_count, 0) # 특수문자 제거
        like_counts.append(like_count) # 리스트에 추가

        # 가사
        lyric = str(l_soup.select(
            '#container > section.sectionPadding.contents.lyrics > div.innerContainer > div.lyricsContainer > p > xmp'))
        # 가사 데이터가 있는 html 구조 #container > section.sectionPadding.contents.lyrics > div > div > xmp
        lyric = re.sub('<.+?>', '', lyric, 0)
        song_lyrics.append(lyric)

    column_list = { # dict-key
        'song_id': id_list,
        'song_name': song_title,
        'artist': song_artist,
        'album': album_list,
        'Like_Count': like_counts,
        'Lyric': song_lyrics,
    }

    df = pd.DataFrame.from_dict(column_list, orient='index') # dict를 dataframe 형식으로 변환
    df = df.transpose() # 행과 열 전환

    return df.to_csv(f'data/{artist_id}.csv') # 편집자 실행


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


## Album Scraping

In [15]:
def album_scraping(artist_id_):
    album_name = []
    artist_list = []
    album_release_date = []
    album_id_final = []
    album_id_link = []
    artist_id = [] 

    #container > section > div > ul > li:nth-child(1)
    url = f'https://music.bugs.co.kr/artist/{artist_id_}/albums'
    request = requests.get(url)
    html = request.text
    soup = BeautifulSoup(html, "html.parser")

    album_list = soup.find("ul", attrs={'class':'list tileView albumList'})

    al_title = soup.select('div.albumTitle')
    ar_title = soup.select('p.artist') # p 태그의 클래스가 artist인 데이터
    al_date = soup.select('p:nth-child(2) > time') 


    al_title_ = soup.select('div.albumTitle > a')

    for html in al_title_:
        album_name.append(html['title'])

    url = f'https://music.bugs.co.kr/artist/{artist_id_}/albums'
    request = requests.get(url)
    html = request.text
    soup = BeautifulSoup(html, "html.parser")

    for elem in soup.find_all('a', href=re.compile('https://music.bugs.co.kr/album/')):
        album_id_link.append(elem['href']) # a 태그 중에서 href 데이터만 따로 빼내서 id_url_list에 담기

    album_id_list_wan = []
    for j in range(len(album_id_link)):
        a = album_id_link[j].lstrip('https://music.bugs.co.kr/album/')  # 링크1 제거
        a1 = a.rstrip('wl_ref=list_ab_03_ab')
        a2 = a1.rstrip('wl_ref=list_ab_01')
        b = a2.rstrip('?')  # 물음표 제거

        album_id_list_wan.append(b)

    album_id_final = list(set(album_id_list_wan))


    for i in range(len(album_id_final)):

        album_date = al_date
        album_release_date.append(al_date[i].get_text())
    
    for i in range(len(album_name)):
        ar_name_ = ar_title[i].get_text().strip('\n')
        artist_list.append(ar_name_)
    
    for _ in range(len(album_name)):
        artist_id.append(artist_id_)
        
    column_list = {
        'artist_id': artist_id,
        'artist': artist_list,
        'album': album_name,
        'album_id' : album_id_final,
        'date': album_release_date
    }

    

    df1 = pd.DataFrame.from_dict(column_list, orient='index') # dict를 dataframe 형식으로 변환
    df1 = df1.transpose() # 행과 열 전환 

    return df1.to_csv(f'data/{artist_id_}_album.csv') # 편집자 실행


In [21]:
indie_id_2 = indie_id_list[140:]


In [ ]:
%%time
for i in indie_id_2:
    lyrics_scraping(i)
    album_scraping(i)

## Merge

In [14]:
def merge_df(artist_id_):
    ar_df = pd.read_csv(f'data/{artist_id_}.csv')
    al_df = pd.read_csv(f'data/{artist_id_}_album.csv')
    merged_df = ar_df.merge(al_df, how='inner', on='album')
    merged_df = merged_df[['artist_id', 'artist_x','song_id', 'song_name', 'album_id', 'album', 'Like_Count', 'Lyric', 'date']]
    return merged_df.to_csv(f'data/merged/{artist_id_}_merged.csv')

In [15]:
for i in indie_id_list:
    merge_df(i)

## Concat (All artists)

In [16]:
df_list2 = []
for i in indie_id_list:
    df_list2.append(pd.read_csv(f'data/merged/{i}_merged.csv'))

In [23]:
all_df = pd.concat(df_list2)
all_df.dropna(inplace = True)
len(all_df)

3094

In [25]:
all_df.drop(all_df.columns[all_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [28]:
all_df.rename(columns={'artist_x': 'artist_name'}, inplace=True)

In [18]:
all_df.to_csv('data/indie_final_dataframe.csv')

In [30]:
all_df.to_csv('indie_final_df.csv')